<a href="https://colab.research.google.com/github/jx-dohwan/Aiffel_EGLMS_Project/blob/main/%5BExp_10%5DLet's_make_a_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Exp_10]Let's make a translator

## 1.import 및 데이터로드

### 1) Import

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

### 2) 데이터로드

In [3]:
file_path = "/content/drive/MyDrive/인공지능/아이펠/ES/data/translator_seq2seq/fra.txt"
lines = pd.read_csv(file_path, names=['eng', 'fra'], sep='\t')
print('전체 샘플수:',len(lines))
lines.sample(5)

전체 샘플수: 194513


,eng,fra
My name is Tom and I'll be your server tonight.,Je m'appelle Tom et je serai votre serveur ce ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
I'm always happy.,Je suis toujours heureux.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Tom can't control himself.,Thomas n'arrive pas à se contrôler.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
Get out your notebooks and pens.,Sortez vos cahiers et vos stylos.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
I promised I wasn't going to do this.,J'ai promis que je n'allais pas faire ça.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...



데이터에서 상위 33,000개의 샘플만 사용해주세요.

33000개 중 3000개는 테스트 데이터로 분리하여 모델을 학습한 후에 번역을 테스트 하는 용도로 사용합니다

In [4]:
lines = lines[['eng', 'fra']][:33000]
lines.sample(5)

,eng,fra
Duck!,Baisse-toi !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Bring me my cane.,Apportez-moi ma canne.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
I can't do that.,Je n'arrive pas à le faire.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
They're doing fine.,Ils vont bien.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
You don't say.,Vraiment ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


## 2. 정제, 정규화, 전처리
- 구두점(Punctuation)을 단어와 분리
- 소문자로 change
- 띄어쓰기 단위로 토큰을 수행

## 3. 디코더의 문장에 시작 토큰과 종료 토큰 입력

## 4. 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요
- https://wikidocs.net/31766

## 5. 임베딩 층(Embedding layer) 사용하기
- https://wikidocs.net/33793
- 주의할 점 : 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만 디코더의 훈련 과정과 테스트 과정에서의 임베딩층은 동일해야 한다.

## 6. 모델 구현하기

## 7. 모델 평가하기

## 8. 회고